In [1]:
import torch


In [7]:
a = torch.ones((2,4))

In [8]:
a

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [13]:
a = a.new_ones(6,5,dtype=torch.half)

In [14]:
a.size()

torch.Size([6, 5])

In [27]:
b = torch.rand(3,3,3)

In [21]:
result = torch.empty(3,3)

In [22]:
result

tensor([[ 1.5080e-38,  0.0000e+00, -2.0318e+27],
        [ 4.4504e-41, -2.0245e+27,  4.4504e-41],
        [-4.3768e+28,  4.4504e-41,  1.5081e-38]])

In [28]:
  b

tensor([[[0.9193, 0.7425, 0.6969],
         [0.7940, 0.3751, 0.8684],
         [0.2571, 0.4118, 0.2587]],

        [[0.3523, 0.8755, 0.7364],
         [0.0662, 0.1382, 0.7688],
         [0.3456, 0.2302, 0.0432]],

        [[0.6057, 0.3117, 0.9829],
         [0.6973, 0.3240, 0.5268],
         [0.3877, 0.3314, 0.2726]]])

In [30]:
b[:,2:3]

tensor([[[0.2571, 0.4118, 0.2587]],

        [[0.3456, 0.2302, 0.0432]],

        [[0.3877, 0.3314, 0.2726]]])

In [31]:
b = b.numpy()

In [32]:
type(b)

numpy.ndarray

### Autograd -- automati differentiation
definition - calculates derivatives ,records graph for all the operations performed on gradient tensor .It is reverse mode actomatic differentiation ,i.e for computing gradient.

In [35]:
a = torch.ones(2,2,requires_grad=True)
a

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [34]:
b = a + 2
print(b)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [36]:
print(b.grad_fn)

In [40]:
c = b*b*3
out = c.mean()
print(c,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [42]:
p = torch.randn(3,3)
p = ((p*3)/(p-1))
print(p.requires_grad)

False


In [44]:
p.requires_grad_(True)
print(p.requires_grad)

True


In [45]:
q = (p*p).sum()
print(q.grad_fn)

In [46]:
out

tensor(27., grad_fn=<MeanBackward0>)

In [47]:
out.backward()

In [50]:
a = torch.ones(2,2,requires_grad=True)

In [55]:
print(a.grad)

None


In [51]:
print(a.grad)


None


### Define a neural network

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [60]:
class network(nn.Module):
  def __init__(self):
     super(network, self).__init__()

     self.conv1 = nn.Conv2d(1,6,3) # 1-- input dim / 6 -- output dim / 3 -- kernal or filter size(3 * 3)
     self.conv2 = nn.Conv2d(6,16,3)

     self.fc1 = nn.Linear(16*6*6,120)
     self.fc2 = nn.Linear(120,84)
     self.fc3 = nn.Linear(84,10)


  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x = x.view(-1,self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  def num_flat_features(self,x):
    size = x.size()[1:]
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

net = network()
print(net)



network(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [63]:
list(net.parameters())[0]

Parameter containing:
tensor([[[[-0.0541,  0.3060,  0.2261],
          [ 0.0035, -0.0291, -0.1247],
          [ 0.1093,  0.0178, -0.1798]]],


        [[[ 0.2942, -0.2039,  0.0541],
          [-0.0542,  0.1772,  0.2888],
          [-0.0193, -0.0057, -0.0160]]],


        [[[ 0.2686, -0.0562, -0.1927],
          [-0.0760, -0.3111, -0.2663],
          [ 0.0365, -0.0761, -0.2804]]],


        [[[-0.0779, -0.2778,  0.2931],
          [-0.1957, -0.0495, -0.2740],
          [-0.1555, -0.1132,  0.0644]]],


        [[[ 0.0930, -0.0497,  0.1184],
          [ 0.0633,  0.2020, -0.0932],
          [ 0.2301,  0.0701,  0.1825]]],


        [[[ 0.2181,  0.1294, -0.2584],
          [ 0.0109,  0.1026,  0.2146],
          [ 0.1458, -0.3326, -0.1145]]]], requires_grad=True)

In [61]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # learneable parameters of model are returns by net.parameters()

10
torch.Size([6, 1, 3, 3])


In [64]:
inp = torch.randn(1,1,32,32)
out = net(inp)
print(out)

tensor([[ 0.0263, -0.0391, -0.0737,  0.0346, -0.0766, -0.1009, -0.0039,  0.0266,
         -0.0158,  0.0019]], grad_fn=<AddmmBackward0>)


In [65]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [66]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [67]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


In [68]:
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

In [69]:
y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)

tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])

In [70]:
tensor

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])

In [72]:
gg = tensor.sum()
gg

tensor(12.)

In [75]:
agg_item = gg.item()
agg_item

12.0

In [76]:
tensor.add_(5)

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])

* tensor to numpy

In [77]:
n = tensor.numpy()
n

array([[6., 5., 6., 6.],
       [6., 5., 6., 6.],
       [6., 5., 6., 6.],
       [6., 5., 6., 6.]], dtype=float32)

In [79]:
tensor.add_(1)

tensor([[7., 6., 7., 7.],
        [7., 6., 7., 7.],
        [7., 6., 7., 7.],
        [7., 6., 7., 7.]])

* numpy array to tensor

In [82]:
t = torch.from_numpy(n)
t


tensor([[7., 6., 7., 7.],
        [7., 6., 7., 7.],
        [7., 6., 7., 7.],
        [7., 6., 7., 7.]])

In [83]:
%%time
value = 0
for i in range(len(tensor)):
  value += tensor[i] * tensor[i]
value

CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 1.13 ms


tensor([196., 144., 196., 196.])